In [1]:
import pygame
import sys
#import ctypes

import numpy as np
from PIL import Image

import tensorflow as tf
from tensorflow import keras
tf.function(reduce_retracing = True)

def bar_graph(probabilities):
    line_margin = 40
    guess_margin = 60
    
    line_x = window_width - 200 + line_margin - 1
    pygame.draw.line(canvas, (0, 0, 0), (line_x, margin + guess_margin), (line_x, window_height - margin))
    line_height = (window_height - 2*margin) - guess_margin
    
    for i in range(10):
        # bars
        bar_length = 150 * probabilities[i]
        bar_height = (line_height)/13
        bar_left = window_width - 200 + line_margin
        bar_top = margin + guess_margin + i * bar_height * 4/3
        
        if i < 9: pygame.draw.rect(canvas, (0, 0, 0), ((bar_left, bar_top), (bar_length, bar_height)))
        # fraction correction
        else: pygame.draw.rect(canvas, (0, 0, 0), ((bar_left, bar_top + 2), (bar_length, bar_height)))
        
        # percentages
        percentage = str(int(round(probabilities[i], 2)*100)) + "%"
        render_text(14, False, percentage, (0, 0, 0), "right", (line_x - 2, bar_top + bar_height*4/6))
        
        # labels
        render_text(14, False, str(labels[i]), (0, 0, 0), "center", (window_width - 200 + 100, bar_top - 1))

def prep_image():
    im = Image.open('images/center.png')
    
    # get rid of opacity
    im = im.convert("L")
    
    # resize
    im = im.resize((28, 28))

    # convert to array
    pixels = np.asarray(im)[None, :]

    #inversion
    average_pixel_value = 0
    for i in range(28):
        for j in range(28):
            average_pixel_value += pixels[0][i][j]
    
    average_pixel_value /= 28*28
    if average_pixel_value > 255/2:
        pixels = 255-pixels

    # input image
    input = Image.fromarray(pixels[0])
    input.save('images/ai_input_image.png')

    pixels = pixels.astype('float32')
    pixels /= 255
    
    return pixels 

def get_prediction(pixels):
    return model.predict(pixels, verbose = 0)

def get_max_label(probabilities):
    if(probabilities[0] == probabilities[1] == probabilities[2] == probabilities[3] == probabilities[4] == probabilities[5] 
       == probabilities[6] == probabilities[7] == probabilities[8] == probabilities[9]): return "uncertain"
    
    max_value = 0
    max_index = 0
    for i in range(len(probabilities)):
        if probabilities[i] > max_value:
            max_value = probabilities[i]
            max_index = i
            
    return str(labels[max_index])
    
def render_text(size, bolded, text, color, side, pos):
    font = pygame.font.SysFont('Times New Roman', size, bolded)
    text = font.render(text, True, color)
    textRect = text.get_rect()
    if side == "left": textRect.bottomleft = pos
    elif side == "center": textRect.midbottom = pos
    elif side == "right": textRect.bottomright = pos
    
    canvas.blit(text, textRect.topleft)

def fill(x, y):
    # if x and y are onscreen
    if x < 0 or x >= 600 or y < 0 or y >= 599:
        return

    # if it's black
    if canvas.get_at((x, y)) == (0, 0, 0):
        return

    center.set_at((x, y), (0, 0, 0))

    # fill the neighboring positions
    try:
        fill(x - 1, y)
        fill(x + 1, y)
        fill(x, y - 1)
        fill(x, y + 1)
    except RecursionError:
        return

#!!!!!!!
#resizing
#fill with BFS



# pygame setup
pygame.init()
# ctypes.windll.shcore.SetProcessDpiAwareness(True)
icon = pygame.Surface((1, 1))
icon.fill((255, 255, 255))
pygame.display.set_icon(icon)
pygame.display.set_caption("Art")

screen = pygame.display.Info()
window_width, window_height = (screen.current_w * 2/3, screen.current_h * 6/7)
canvas = pygame.display.set_mode((window_width, window_height), pygame.RESIZABLE)
center = pygame.Surface((window_width-200, window_height))
center.fill((255, 255, 255))
sidebar = pygame.Surface((200, window_height))
margin = 10


# variables
clock = pygame.time.Clock()

drawing = False
do_interpolate = False;
radius = 20
painteraser = True

model = keras.models.load_model('dn/dn.h5')
labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
probabilities = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


running = True
while running:
    clock.tick(1000)

    # events
    for event in pygame.event.get():
        # window x button
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.VIDEORESIZE:
            if(canvas.get_size()[0] > 200 and canvas.get_size()[1] > 0):
                window_width, window_height = canvas.get_size()
                window_width, canvas_height = (window_width-200, window_height)

        if event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == pygame.BUTTON_LEFT:
                drawing = True

            if event.button == pygame.BUTTON_RIGHT:
                try:
                    pygame.image.save(center, "images/center.png")
                    probabilities = (get_prediction(prep_image())[0]).tolist()
                    ai_input_image = pygame.image.load('images/ai_input_image.png')
                except(pygame.error): pass

        if event.type == pygame.MOUSEBUTTONUP:
            if event.button == pygame.BUTTON_LEFT:
                drawing = False
                do_interpolate = False

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                center.fill((255, 255, 255))
            if event.key == pygame.K_p:
                painteraser = True
                radius -= 10
            if event.key == pygame.K_e:
                painteraser = False
                radius += 10
            if event.key == pygame.K_f:
                fill(pygame.mouse.get_pos()[0], pygame.mouse.get_pos()[1], )
            
            if event.key == pygame.K_EQUALS:
                radius += 1
            if event.key == pygame.K_MINUS:
                radius -= 1
            
    
    #---------------Borderlayout.CENTER---------------#
    if drawing:
        if painteraser: color = (0, 0, 0)
        else: color = (255, 255, 255)

        cx = pygame.mouse.get_pos()[0]
        cy = pygame.mouse.get_pos()[1]
        pygame.draw.circle(center, color, (cx, cy), radius, 0)

        if do_interpolate:
            changeX = cx - px
            changeY = cy - py
            if changeX == 0:
                for i in range(min(cy, py), max(cy, py), 1):
                    pygame.draw.circle(center, color, (cx, i), radius, 0)
            else:
                slope = changeY / changeX
                b = cy - slope * cx

                if abs(changeX) > abs(changeY):
                    for i in range(min(cx, px), max(cx, px), 1):
                        pygame.draw.circle(center, color, (i, int(slope * i + b)), radius, 0)
                else:
                    for i in range(min(cy, py), max(cy, py), 1):
                        pygame.draw.circle(center, color, (int((i - b) / slope), i), radius, 0)

        px = cx
        py = cy
        do_interpolate = True
    
    canvas.blit(center, (0, 0))

    #---------------Borderlayout.EAST---------------#
    sidebar.fill((200, 200, 200))
    canvas.blit(sidebar, (window_width-200, 0))

    # guess
    render_text(14, False, get_max_label(probabilities), (0, 0, 0), "center", (window_width-200 + 100, 20))
    # full output
    bar_graph(probabilities)
    
    # input image
    try:
        input_image = pygame.image.load('images/ai_input_image.png')
        canvas.blit(input_image, (window_width-200 + 50, 40))
    except(FileNotFoundError): pass
    

    # canvas
    pygame.draw.line(canvas, (0, 0, 0), (0, 0), (window_width, 0))
    pygame.draw.line(canvas, (0, 0, 0), (window_width-200, 0), (window_width-200, window_height))

    pygame.display.flip()


pygame.quit()
quit()

pygame 2.5.1 (SDL 2.28.2, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


: 